## 데이터 준비하기

이제 


In [ ]:
using Downloads, DataFrames, CSV, Chain, Dates

url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
file = Downloads.download(url)
df = CSV.File(file) |> DataFrame

In [ ]:
kor = @chain df begin
    filter([:iso_code] => (iso_code) -> iso_code == "KOR", _)
end;

In [ ]:
first(kor, 5)

In [ ]:
last(kor, 5)

In [ ]:
using Plots, StatsPlots, LaTeXStrings

@df kor plot(:date,
    :new_cases,
    xlab=L"t", ylab="infected daily",
    yformatter=y -> string(round(Int64, y ÷ 1_000)) * "K",
    label=false)

In [ ]:
function NegativeBinomial2(μ, phi)
    p = 1 / (1 + μ / phi)
    r = phi

    return NegativeBinomial(r, p)
end

In [ ]:
using DifferentialEquations

function sir_ode!(du, u, p, t)
    (S, I, R) = u
    (β, γ) = p
    N = S + I + R    # N은 전체 인원입니다.
    infection = β * I * S / N 
    recovery = γ * I
    @inbounds begin
        du[1] = -infection # u의 첫번째 아이템의 전미분인 미감염자의 비율입니다.
        du[2] = infection - recovery # Infected
        du[3] = recovery # Recovered
    end
    nothing
end;

In [ ]:
using Turing
using LazyArrays
using Random: seed!

seed!(123)

@model function bayes_sir(infected, i_, r_, N)
    #calculate number of timepoints
    l = length(infected)

    #priors
    β ~ TruncatedNormal(2, 1, 1e-4, 10)     # using 10 because numerical issues arose
    γ ~ TruncatedNormal(0.4, 0.5, 1e-4, 10) # using 10 because numerical issues arose
    phi_ ~ truncated(Exponential(5); lower=0, upper=1e5)
    phi = 1.0 / phi_

    #ODE Stuff
    I = i_
    u0 = [N - I, I, r_] # S,I,R
    p = [β, γ]
    tspan = (1.0, float(l))
    prob = ODEProblem(sir_ode!,
        u0,
        tspan,
        p)
    sol = solve(prob,
        Tsit5(), # similar to Dormand-Prince RK45 in Stan but 20% faster
        saveat=1.0)
    sol_ = Array(sol)[2, :] # New Infected
    sol_ = max.(1e-4, sol_) # numerical issues arose

    #likelihood
    infected ~ arraydist(LazyArray(@~ NegativeBinomial2.(sol_, phi)))
end;

In [ ]:
infected = kor[:, :new_cases]
r_ = 0
i_ = 1
N = maximum(kor[:, :population])
model_sir = bayes_sir(infected, i_, r_, N)
chain_sir = sample(model_sir, NUTS(), 1_000)
summarystats(chain_sir[[:β, :γ]])